In [10]:
#Load related libraries to do analysis
import pandas as pd #To work with dataset
import numpy as np #Math library

In [2]:
## III.2.1 Persiapan data dan impor ke Python

# library panda untuk menggunakan data frame
import pandas as pd

# Pengaturan folder tempat data / Google Colab

# Impor data dari excel xls ke Python
df = pd.read_excel('input/german_credit_easy.xls')

In [18]:
## III.3.1 Penanganan untuk data yang hilang (missing values)

# Membuat kopi dari df dan menyimpan sebagai df_prepare
df_prepare = df.copy()

In [19]:

# Koding berikut menggantikan missing value dari semua variabel dengan
# nilai yang paling sering muncul (modus)

for column in df_prepare.columns:
        df_prepare[column].fillna(df_prepare[column].mode()[0], inplace=True)

In [5]:
## III.3.3 Pembuatan variabel yang baru (Feature Engineering) 

df_prepare['cicilan_per_bulan'] = df_prepare['credit_amount'] / df_prepare['duration_in_month']

In [21]:
df_prepare.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   ID                          1000 non-null   object
 1   default                     1000 non-null   int64 
 2   account_check_status        1000 non-null   object
 3   duration_in_month           1000 non-null   int64 
 4   credit_history              1000 non-null   object
 5   purpose                     1000 non-null   object
 6   credit_amount               1000 non-null   int64 
 7   savings                     1000 non-null   object
 8   present_emp_since           1000 non-null   object
 9   installment_as_income_perc  1000 non-null   int64 
 10  personal_status_sex         1000 non-null   object
 11  other_debtors               1000 non-null   object
 12  present_res_since           1000 non-null   int64 
 13  property                    1000 non-null   objec

In [26]:
# Create new variable using def function
# Note that this will be slow with big dataframe
def category_duration(row):
    if (row['duration_in_month'] < 0):
        val = '0.Negative?'
    elif (row['duration_in_month'] >= 24) & (row['duration_in_month'] <= 36):
        val = '1.24<x<=36mth'
    elif (row['duration_in_month'] < 24) | (row['duration_in_month'] > 36):
        val = '2.<24or>36mth'
    else:
        val = '3.WrongNumber' #default value
    return val
df_prepare['category_duration'] = df_prepare.apply(category_duration, axis = 1)

In [25]:
# Create new variable using numpy operation
# This will be very fast even with big dataframe
conditions =    [
                (df_prepare['duration_in_month'] < 0),
                ((df_prepare['duration_in_month'] >= 24) & (df_prepare['duration_in_month'] <= 36)),
                ((df_prepare['duration_in_month'] < 24) | (df_prepare['duration_in_month'] > 36))
                ]
choices =       [
                '0.Negative?',
                '1.24<x<=36mth',
                '2.<24or>36mth'
                ]
df_prepare['category_duration2'] = np.select(conditions,choices, default=np.nan) #nan is default value

In [6]:
## III.3.4 Transformasi variabel kategori menjadi numerik (One-Hot Encoding) 

# Koding untuk one hot encoding untuk semua variabel kategori

# Langkah pertama adalah membuang variabel ID karena ini adalah identifikasi nasabah
df_prepare.drop(['ID'], axis = 1, inplace = True)

# Memisahkan variabel kategori dengan variabel numerik
dataset_dummies = df_prepare.select_dtypes(exclude = ['float64','int64','int'])
dataset_int = df_prepare.select_dtypes(include=['float64','int64','int'])

# Memisahkan target/dependent variabel dari dataset dengan variabel kategori
target = df_prepare['default']
dataset_int.drop(['default'], axis = 1, inplace = True)

# Menggunakan function get_dummies dari panda untuk melakukan one hot encoding
dataset_dummies_df = pd.get_dummies(dataset_dummies,drop_first = True)
    
# Menggabungkan dataset variabel kategori dari one hot encoding dengan variabel numerik
dataset = pd.concat([dataset_dummies_df, dataset_int], axis = 1)
feature_name = dataset.columns
df_encoded = pd.concat([dataset, target], axis = 1)

In [27]:
# Subset dataframe by criteria
example1 = df_encoded[df_encoded.default == 0]

In [29]:
# Only select some columns from dataframe
example1 = df_prepare[['default','category_duration2']]

In [12]:
df_encoded.to_pickle('output/df_encoded.pkl')